# Positioning the detectors for the grating spectrograph

In [ ]:
from nbtemplate import display_header, get_path
display_header('Placethedetector.ipynb')

## Goal

Currently, there are two instruments planned for Lynx that need to be located at the focal point. One of them is a microcalorimeter, which is mounted in a dewar. For the grating spectrometer, we need a separate detector to catch the dispersed light. Because of the dewar, this detector cannot reach down close to the zeroth order. 

We design the spectrometer with critical angle transmission (CAT) gratings, which are operated with a certain blaze angle. Most of the dispersed signal is found close to twice the blaze angle in a region called the blaze peak. The width of this peak depends on the grating properties (such as the dimensions of the grating bars) and the distribution of blaze angles (due to the finite size of the gratings and the finite size of the mirror PSF not all rays hit the gratings at the blaze angle chosen in the design).

In this notebook, I simulate our fiducial design for the spectrometer and analyze which fraction of the photons can be detected for a detector of a given size and position.

In the last section, I also look at the signal seen in the zeroth order for a range of energies.

In [ ]:
import sys
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.table import Table, QTable, join
import astropy.units as u
import marxs
from marxs import visualization

from marxs.source import PointSource, FixedPointing, JitterPointing
from marxs.analysis import resolvingpower_from_photonlist
from marxs.simulator import Sequence

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from marxslynx import lynx

In [ ]:
wave = np.arange(12., 50., .1) * u.Angstrom
energy = wave.to(u.keV, equivalencies = u.spectral())
flux = np.ones(len(wave))[1:] / np.abs(np.diff(energy))

mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                       energy=Table({"energy": energy[::-1][1:], 
                                     "fluxdensity": flux[::-1] / u.cm**2 / u.s}),
                       flux=1. / u.s / u.cm**2)
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(1e5 * u.s)
photons = fixedpointing(photons)

In [ ]:
pl = lynx.PerfectLynx()

photons = pl(photons)
photons = photons[np.isfinite(photons['order'])]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ind = (photons['detcirc_phi'] > -0.1) & (photons['detcirc_phi'] < 0.3)
out = ax.hist(photons['detcirc_phi'][ind], weights=photons['probability'][ind], bins=50)
ax.set_ylabel('Number of photons / bin')
ax.set_xlabel(r'$\varphi$ (rad)')

Using a simulation with a flat input spectrum (in wavelength space), this figure shows the angular distribution of photons. The x-coordinate of the plot is the angle measured on a cylindrical detector along the Rowland circle.

In [ ]:
photons['wave'] = photons['energy'].to(u.Angstrom, equivalencies=u.spectral())
wavelim = np.arange(12., 49, 12)
px = [photons['proj_x'][(photons['wave'] > wavelim[i]) & (photons['wave'] < wavelim[i+1])] for i in range(len(wavelim)-1)]
weights = [photons['probability'][(photons['wave'] > wavelim[i]) & (photons['wave'] < wavelim[i+1])] for i in range(len(wavelim)- 1)]
labels = ['{}-{} nm'.format(wavelim[i]/10, wavelim[i+1]/10) for i in range(len(wavelim) -1)]

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
out = ax.hist(px, weights=weights, bins=np.arange(-400, 800, 30), histtype='barstacked', label=labels)
ax.legend()
ax.set_ylabel('Number of photons')
ax.set_xlabel('Distance from focal point [mm]')
fig.savefig(get_path('figures')+ '/detectorplacement.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/detectorplacement.pdf', bbox_inches='tight')


This is a very similar to the plot above. This time, the x-axis of the plot is simply the distance of the detected position projected onto the focal plane. The blaze peak is wider for photons with a longer wavelength. From this figure, we cal already read off that we need to cover about the region from 400 to 700 mm measured from the zeroth order to collect the bulk of the dispersed signal.

In [ ]:
fig = plt.figure(figsize=(8, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i in np.arange(-9, 1):
    porder = photons[photons['order'] == i]
    out = ax1.hist(porder['proj_x'], weights=porder['probability'], bins=np.arange(-400, 800, 25.), 
                   label=f'{i}', histtype='step')
    out = ax2.hist(porder['wave'], weights=porder['probability'], bins=np.arange(5., 50., 1.),
                  histtype='step')

ax1.legend(loc='upper left')
ax1.set_ylabel('Number of photons')
ax1.set_xlabel('Distance from focal point [mm]')
ax2.set_xlabel('wavelength [Ang]')
ax2.set_ylabel('Number of photons')

*left:* Same plot as above, but separated by dispersion order. *right:* Most wavelength will be dispersed into more than one order and each order is relevant for a range of wavelength.

## The zeroth order

The presence of the CAT gratings reduces the signal detected in the zeroth order. Some photons are dispersed to higher orders (after all, that is why we use the gratings in the first place), but there is also a loss due to absorption by the grating frames, the mounting structure, the support structure that is part of the grating membrane, and also the grating bars themselves. For high energies, Si becomes transparent, but the grating still disperse some signal because they act as phase shifting gratings at that point.

On the other hand, the signal detected in the zeroth order is scientifically valuable. First, an accurate wavelength calibration depends on the position of the zeroth order, and second, the microcalorimeter offers a better resolving power at higher energies than the CAT gratings, not to mention a much higher signal. If sufficient light passes through the gratings, soft X-rays can be analyzed in the grating spectrometer at the same time as the high-energy photons are analyzed in the microcalorimeter.

In [ ]:
from marxs import optics
from marxslynx.lynx import conf


In [ ]:
from marxslynx.lynx import RowlandDetArray
conf['det_kwargs']['theta'] = [3.1, 3.18]
r = RowlandDetArray(conf)

In [ ]:
energybins = np.arange(.1, 10, .1) * u.keV
energymidpoints = 0.5 * (energybins[:-1] + energybins[1:])

In [ ]:
mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                           energy=QTable({"energy": energybins, 
                                   "fluxdensity": np.ones(len(energybins)) / u.s / u.cm**2 / u.keV}),
                      )
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(2e5 * u.s)
photons = fixedpointing(photons)

In [ ]:
# My current efficieny table for the mirror stops at 3 keV. 
# In order to simulate anything at higher energy, I need to extend that range.
# I make two assumptions here. Both are clearly wrong, but should be sufficienct for
# a simple estimate of the relative loss of photons at high energies.
from marxslynx.mirror import metashelleff, metashellgeometry, MetaShellEfficiency
from scipy.interpolate import interp1d

lastrow = pl.elements[1].elements[1].relative_eff[:, -1].reshape(12, 1)
releff = np.hstack([pl.elements[1].elements[1].relative_eff, 
                    lastrow, lastrow])
energy = np.hstack([metashelleff['energy'], 3.1, 12.])
extendedrangeshell = MetaShellEfficiency()
extendedrangeshell.shells = [(i, interp1d(energy, releff[i - 1, :]))
                              for i in metashellgeometry['Metashell Serial Number']]
pl.elements[1].elements[1] = extendedrangeshell

In [ ]:
photons = pl.elements[0](photons)
photons = pl.elements[1](photons)

In [ ]:
pl.elements[2].elements[0].elements[0]

In [ ]:
# Now run with uncoated gratings
from marxs.optics.grating import CATGrating
from marxslynx.ralfgrating import order_selector_Si

for e in pl.elements_of_class(CATGrating):
    e.order_selector = order_selector_Si
    
p = pl.elements[2](photons.copy())
p = pl.elements[3](p)
p = pl.elements[4](p)

In [ ]:
# Now run the same with Pt coated gratings
from marxslynx.ralfgrating import order_selector_SiPt

for e in pl.elements_of_class(CATGrating):
    e.order_selector = order_selector_SiPt

ppt = pl.elements[2](photons.copy())
ppt = pl.elements[3](ppt)
ppt = pl.elements[4](ppt)

In [ ]:
before = np.histogram(photons['energy'], weights=photons['probability'], bins=energybins)
ind = p['order'] == 0
after = np.histogram(p['energy'][ind], weights=p['probability'][ind], bins=energybins)
ind = ppt['order'] == 0
afterpt = np.histogram(ppt['energy'][ind], weights=ppt['probability'][ind], bins=energybins)

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
frac = after[0] / before[0]
fracpt = afterpt[0] / before[0]
tsubaperangle = np.linspace(0, np.pi/2, 7)[1:]
for i, ang in enumerate(tsubaperangle):
    line, = ax.plot(energymidpoints, 
             1 - ((2 * ang)/np.pi * frac + (np.pi -(2*ang))/np.pi), 
             label='{:3.0f} %'.format(np.rad2deg(ang)*4/360*100))
    ax.plot(energymidpoints, 
            1 - ((2 * ang)/np.pi * fracpt + (np.pi -(2 * ang))/np.pi), 
            ls=':', color=line.get_color())
ax.legend(title='Aperture area\ncovered\nby gratings', ncol=1, loc='upper right')
ax.set_xlabel('energy [keV]')
ax.set_ylabel('fraction of photons removed from\nbeam when XGS is inserted')
ax.set_ylim([0, 1.])
ax.set_xlim([0, 16.])
fig.savefig(get_path('figures') + '/highen.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/highen.pdf', bbox_inches='tight')

This plot shows how much of the incoming signal is removed from the beam by the CAT gratings. The solid lines are for pure Si gratings, the dotted lines for Si gratings coated with Pt. This increases the grating efficiency between 1 and 2 keV, but is also reduces the signal seen in zeroth order. Different colors represent different filling factor. At this point, we do not take into account how the effective area of the mirror changes from the center to the outer shells. Still, even in this simple approximation, 70% of the high-energy signal would still be available for the microcalorimeter, even if we cover 2/3 of the aperture with CAT gratings. This fraction decreases for lower energies, in particular when using PT coated gratings, but below about 2 keV the dispersed signal in the grating spectrometer is more valuable than the zeroth-order signal anyway. Unless the entire aperture is filled with gratings, there will still be sufficient signal at the zeroth order to determine the position, even for very soft sources.